
# RAG Lab 2: Query → Retrieve (Chroma) → Rerank → HF LLM Answer

**Updated:** 2025-11-08

In this short lab you will:
1. Enter a natural-language **query** via a widget
2. **Embed** the query using the **same model** as your document embeddings
3. Retrieve **top k×2** candidates from an **existing ChromaDB** collection
4. **Rerank** those candidates with a **cross-encoder** reranker
5. Keep the **top k** passages and build a grounded prompt
6. Run **Hugging Face LLM** inference and display the **answer**

> Goal: illustrate a lean RAG pipeline with reranking. 


In [ ]:
# ✅ Install minimal dependencies. If these are already installed, this cell is a no-op.
# !pip install langchain langchain-community pypdf chromadb sentence-transformers transformers tqdm --quiet

Python: 3.11.13 | Platform: Linux-5.14.0-570.55.1.el9_6.x86_64-x86_64-with-glibc2.34
chromadb: 1.3.2
sentence_transformers: 5.1.2
transformers: 4.57.1
ipywidgets: 8.1.8


In [51]:

# ---- Imports & configuration ----
import os, numpy as np, pandas as pd
from typing import List, Dict
from tqdm import tqdm
import ipywidgets as w

import chromadb
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline

from pprint import PrettyPrinter
# Create a PrettyPrinter with custom indentation
pp = PrettyPrinter(indent=4)

# === Paths & model names (match prior lab defaults) ===
PERSIST_DIR    = "./rag_chroma"                      # where the prior lab persisted vectors
COLLECTION_NAME= "cnu_rag_lab"                       # collection name used previously
EMBED_MODEL_NAME = "sentence-transformers/msmarco-distilbert-cos-v5"
RERANK_MODEL   = "cross-encoder/ms-marco-electra-base"    # compact cross-encoder
LLM_MODEL     = "google/flan-t5-small"               # compact text-generation model
K_DEFAULT      = 5


## 1) Load embedder, reranker and a text generation model to give answers

In [5]:

# Load the SAME embedder used for documents, so query embeddings live in the same space.
embedder = SentenceTransformer(EMBED_MODEL_NAME)
print("Embedder:", EMBED_MODEL_NAME)

# Cross-encoder reranker: computes relevance for (query, passage) pairs. Higher = better.
reranker = CrossEncoder(RERANK_MODEL)
print("Reranker:", RERANK_MODEL)

# Lightweight HF text2text model for grounded answering (fast on CPU relative to larger LLMs).
# You can swap to a chat model later if you have more compute.
gen = pipeline("text2text-generation", model=LLM_MODEL)
print("HF LLM:", LLM_MODEL)


Embedder: sentence-transformers/msmarco-distilbert-cos-v5
Reranker: cross-encoder/ms-marco-electra-base


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


HF LLM: google/flan-t5-small


## 2) Load the chroma database generated in Lab 1

In [6]:
import chromadb

# Connect to existing Chroma collection
client = chromadb.PersistentClient(path=PERSIST_DIR)
try: 
    collection = client.get_collection(name=COLLECTION_NAME)
    print(f"Connected to collection '{COLLECTION_NAME}' with {collection.count()} vectors at {PERSIST_DIR}")
except Exception as e:
    raise SystemExit(
        f"[Error] Could not open Chroma collection '{COLLECTION_NAME}' at {PERSIST_DIR}.\n"
        "Run the previous RAG lab to build it, then re-run this notebook."
    )


Connected to collection 'cnu_rag_lab' with 1239 vectors at ./rag_chroma



## Typical RAG Retrieval Flow (two‑stage)

```
Query  ──► Bi‑encoder vector ──► ANN index (top‑k docs)
                               └─► k candidates
Query + each candidate doc ──► Cross‑encoder (re‑ranker) ──► final ordered list
Top‑m chunks ──► Prompt context for LLM
```



## Why is a re‑ranker more accurate than a bi‑encoder? (Core intuition)

| Model | Input/Scoring | Strengths | Weaknesses |
|---|---|---|---|
| **Bi‑encoder** | Encodes **query** and **doc** *independently* into vectors; score = cosine/dot | Very **fast**, **scalable** (precompute doc vectors; index with FAISS/Chroma/Pinecone) | **No token‑level interaction** between query & doc; can miss subtle meaning (negation, entities, context) |
| **Cross‑encoder** (Re‑ranker) | Reads **query + doc together** (e.g., `[CLS] query [SEP] doc`); predicts **relevance** | **Deep token‑level attention**; **context‑sensitive** scoring; higher **accuracy** | **Slower** (must score each pair), no doc precomputation |

**Example:**

**Query:** “Documents not about CNNs”

**Doc:** “This paper discusses convolutional networks”

- Bi-encoder: high similarity (misses “not”)

- Re-ranker: low relevance (understands negation)


**Reason the re‑ranker wins:** It attends across tokens of query **and** document jointly, so it can model negation, long‑distance dependencies, and nuanced phrasing that a single fixed vector (bi‑encoder output) cannot fully capture.



## 6) Retrieval: Top‑k Nearest Chunks

We'll embed the user query with the **same model** used to create the chroma database in the last lab, then query ChromaDB for the nearest chunks.


In [31]:

def embed_query(q: str) -> np.ndarray:
    '''Embed a query string using the same SentenceTransformer model (normalized).'''
    # For asymmetric semantic search, you are recommended to use SentenceTransformer.encode_query to encode your queries
    v = embedder.encode_query([q], convert_to_numpy=True, normalize_embeddings=True)
    return v[0].astype("float32")

def chroma_retrieve(query: str, k2: int) -> List[Dict]:
    '''Retrieve top k*2 candidates from Chroma using vector similarity.'''
    # q_emb = embed_query(query).tolist()
    q_emb = embed_query(query)

    res = collection.query(
        query_embeddings=[q_emb],
        n_results=k2,
        include=["documents", "metadatas", "distances"]  #returns string chunks, plus info about each page in the pdf and chunk number, and finaly the cosign distance    )
    )
    
    # Flatten results into a list of dicts for easy handling
    items = []
    for cid, doc, meta, dist in zip(res["ids"][0], res["documents"][0], res["metadatas"][0], res["distances"][0]):
        items.append({
            "id": cid,
            "text": doc,
            "source": meta.get("source"),
            "page": meta.get("page"),
            "method": meta.get("method"),
            "distance": float(dist)
        })
    return items

def rerank(query: str, candidates: List[Dict], k: int) -> List[Dict]:
    '''Use a cross-encoder to rerank and return the top-k passages for the final prompt.'''
    pairs = [(query, c["text"]) for c in candidates]
    scores = reranker.predict(pairs)  # vector of relevance scores (higher is better)

    # Attach scores to candidates
    for c, s in zip(candidates, scores):
        c["score"] = float(s)
        
    # Sort by score descending and keep top-k
    return sorted(candidates, key=lambda x: x["score"], reverse=True)[:k]

def build_context(passages: List[Dict]) -> str:
    '''Build a compact context block with source/page for grounding.'''
    lines = []
    for p in passages:
        tag = f"[{p.get('source')} p.{p.get('page')} | {p.get('method')}] "
        text = p["text"].strip().replace("\n", " ")
        lines.append(tag + text)
    return "\n- " + "\n- ".join(lines) if lines else ""


In [33]:
#same function from week11/embeddings_cosine_similarity_mini_lab.ipynb
def build_prompt(passages: List[Dict], question: str) -> dict:

    # context = " ".join(context)  #a list of docs is provided
    context=build_context(passages)

    system_msg = ("""You are a helpful assistant. Answer the user's question **using only** the provided Data.
    If the answer isn't in the context, say you don't know.
    Instructions:
    - Ground your answer in the context.
    - If the answer is not in the context, say "I don't know based on the provided context."
    """)

    user_msg = f"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]
    return messages

## Pass prompt to LLM for answer

In [36]:
from transformers import pipeline
# model_name="text-generation", model="distilgpt2"   # not a chat model
# model_name = "deepset/bert-large-uncased-whole-word-masking-squad2"
# model_name="knowledgator/Qwen-encoder-1.5B"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
llm = pipeline("text-generation", model=model_name,tokenizer=model_name) 


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [57]:

# SYS = "You are a concise TA. Answer ONLY using the provided context. If insufficient, say you don't know. Cite filename and page when possible."

# def answer_with_llm(query: str, passages: List[Dict], max_new_tokens: int = 256, verbose: bool = True) -> str:
#     '''Create a grounded prompt and generate an answer with a small HF model (FLAN-T5).'''
#     context = build_context(passages)
#     # FLAN-T5 is seq2seq; a plain instruction-style prompt works well
#     prompt = (
#         f"{SYS}\n\n"
#         f"Question: {query}\n"
#         f"Context:\n{context}\n\n"
#         f"Answer:"
#     )

#     if(verbose):print(prompt)
#     out = gen(prompt, max_new_tokens=max_new_tokens, num_beams=4, do_sample=False)
#     return out[0]["generated_text"].strip()
def answer_with_llm(query: str, passages: List[Dict], max_new_tokens: int = 256, verbose: bool = True) -> str:
    prompt=build_prompt(passages, query)
    if(verbose):
        pp.pprint(prompt)
    return llm(prompt, max_new_tokens=1000)[0]['generated_text'][2]['content']



#     if(verbose):print(prompt)
#     out = gen(prompt, max_new_tokens=max_new_tokens, num_beams=4, do_sample=False)
#     return out[0]["generated_text"].strip()

In [29]:
# query="what are the pre reqs for CPSC 475"
# chroma_retrieve(query, k2=30)

In [ ]:

# Simple UI: text area for the query and slider for k
q_box   = w.Textarea(value="", placeholder="Type Query Here", description="Query:", layout=w.Layout(width="100%", height="100px"))
k_slider= w.IntSlider(value=K_DEFAULT, min=2, max=10, step=1, description="k")
run_btn = w.Button(description="Retrieve → Rerank → Answer", button_style="primary")
out     = w.Output()

def on_click(_):
    out.clear_output(wait=True)
    query = q_box.value.strip()
    k = int(k_slider.value)
    if not query:
        with out: print("Please type a query.")
        return
    with out:
        print(f'Query length={len(query)}')
        print("Retrieving from Chroma…")
        cands = chroma_retrieve(query, k2=k*10)
        if not cands:
            print("No results. Make sure your Chroma collection is populated (run the first lab).")
            return
        print(f"Retrieved {len(cands)} candidates. Reranking…")
        topk = rerank(query, cands, k=k)
        # Tiny preview table of the top-k reranked passages
        df = pd.DataFrame([{
            "rank": i+1,
            "score": round(p["score"], 4),
            "source": p["source"],
            "page": p["page"],
            "snippet": (p["text"][:220] + "…") if len(p["text"]) > 220 else p["text"]
        } for i, p in enumerate(topk)])
        display(df)
        print("---")
        ans = answer_with_llm(query, topk, max_new_tokens=1000, verbose=False)
        print("---")
        pp.pprint( ans)

run_btn.on_click(on_click)
w.VBox([q_box, k_slider, run_btn, out])

#what are the pre reqs for CPSC 475


### Tips & next steps
- **Why rerankers help:** bi-encoders (embeddings) are fast but approximate; cross-encoders read the *pair* (query, passage) and usually re-order the top candidates more accurately.
- Try other rerankers: `cross-encoder/ms-marco-MiniLM-L-12-v2`, `BAAI/bge-reranker-base` (bigger = slower but better).
- You can swap the LLM (e.g., `TinyLlama/TinyLlama-1.1B-Chat-v1.0`) via `pipeline("text-generation", ...)` if you prefer chat-style models.
- Keep the **same embedder** for documents and queries. Only the reranker changes.
- Experiment: different `k`, prompt templates, or add citation formatting to your final answer.
